# Modelling the world with simpy

You want to simulate a primary care consultation. There are one doctor (Resource) that will attend a patient every 5 minutes, from 10:00 to 13:00. Patients arrive with +/- 5 minutes from their scheduled time. Doctor will take between 4 and 10 minutes to attend each patient. provide commented python code to run the simulation and evaluate the average waiting time and total delay

For this task, we'll use simpy, a discrete-event simulation library in Python.

In [1]:
import simpy
import random

- Initialize the Simulation Environment: We'll use discrete event simulation, where time progresses based on the occurrence of events rather than fixed time steps.

- Define the Resources: In this case, we have one doctor who can attend one patient at a time.

- Create the Patient Arrivals: Patients arrive every 5 minutes, but with a variability of +/- 5 minutes.

- Define the Consultation Process: Each consultation takes between 4 and 10 minutes.

- Collect Statistics: We need to track the average waiting time and total delay.

- Run the Simulation: From 10:00 to 13:00 and analyze the results.



In [2]:
# Setup and run the simulation

env = simpy.Environment()

In [3]:
# Parameters
num_doctors = 1
interval = 5  # Patient arrival interval
wait_times = []  # List to record waiting times
start_time = '10:00'
duration = 3 * 60 # time in minutes


In [4]:
from datetime import datetime, timedelta

In [5]:
def hora(sim_time, start_time=start_time):
    # Parse the time string into a datetime object
    time_obj = datetime.strptime(start_time, '%H:%M')

    # Add the minutes
    new_time = time_obj + timedelta(minutes=sim_time)

    # Format and return the new time
    return new_time.strftime('%H:%M')

In [6]:
def paciente(env, name, doctor, wait_times):
    """Patient process. Each patient will wait for a doctor 
       and then have a consultation."""
    
    arrival_time = env.now
    
    print(f'{name} arrives at {hora(arrival_time)}')
    
    with doctor.request() as request:
        yield request
        wait = env.now - arrival_time  # Calculate the waiting time
        wait_times.append(wait)  # Record the waiting time
        print(f'{name} waited {wait} minutes')
        
        consultation_time = random.randint(4, 10)  # Consultation takes between 4 and 10 minutes
        yield env.timeout(consultation_time)
        print(f'{name} leaves at {env.now}')

In [7]:


def consulta(env, num_doctors, interval, wait_times):
    """Setup the simulation environment."""
    # Create the doctor resource
    doctor = simpy.Resource(env, num_doctors)

    # Generate patients
    i = 0
    while True:
        yield env.timeout(random.randint(interval - 5, interval + 5))  # Patient arrival time variability
        i += 1
        env.process(paciente(env, f'Patient {i}', doctor, wait_times))




In [8]:
# Setup and run the simulation

env.process(consulta(env, num_doctors, interval, wait_times))


<Process(consulta) object at 0x7f005d546ad0>

In [9]:
env.run(until=duration)  # Run for 180 minutes (3 hours from 10:00 to 13:00)

Patient 1 arrives at 10:03
Patient 1 waited 0 minutes
Patient 2 arrives at 10:04
Patient 3 arrives at 10:07
Patient 1 leaves at 12
Patient 2 waited 8 minutes
Patient 4 arrives at 10:16
Patient 5 arrives at 10:16
Patient 6 arrives at 10:16
Patient 2 leaves at 21
Patient 3 waited 14 minutes
Patient 7 arrives at 10:22
Patient 8 arrives at 10:24
Patient 3 leaves at 27
Patient 4 waited 11 minutes
Patient 9 arrives at 10:29
Patient 4 leaves at 32
Patient 5 waited 16 minutes
Patient 10 arrives at 10:34
Patient 5 leaves at 42
Patient 6 waited 26 minutes
Patient 11 arrives at 10:44
Patient 12 arrives at 10:47
Patient 6 leaves at 49
Patient 7 waited 27 minutes
Patient 7 leaves at 53
Patient 8 waited 29 minutes
Patient 13 arrives at 10:57
Patient 14 arrives at 11:00
Patient 15 arrives at 11:00
Patient 8 leaves at 62
Patient 9 waited 33 minutes
Patient 16 arrives at 11:10
Patient 9 leaves at 71
Patient 10 waited 37 minutes
Patient 17 arrives at 11:14
Patient 10 leaves at 78
Patient 11 waited 34 mi

In [10]:
# Calculate and print the average waiting time
average_wait = sum(wait_times) / len(wait_times) if wait_times else 0
print(f'Average waiting time: {average_wait} minutes')

Average waiting time: 38.36 minutes
